In [13]:
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift
from sklearn import preprocessing
style.use('ggplot')

In [14]:
df = pd.read_excel("../titanic.xls")
original_df = pd.DataFrame.copy(df)

In [15]:
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [16]:
df.drop(['body', 'name'], 1, inplace=True)

/var/folders/rw/hql3pd214t59q_rc7_7h0lqm0000gn/T/ipykernel_86804/3798999643.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(['body', 'name'], 1, inplace=True)


In [17]:
df.fillna(0, inplace=True)

In [18]:
def handle_non_numerical_data(df):
    columns = df.columns.values

    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x += 1
            df[column] = list(map(convert_to_int, df[column]))

    return df

In [19]:
df = handle_non_numerical_data(df)
df.head()

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,1,29.0000,0,0,754,211.3375,177,1,1,196
1,1,1,0,0.9167,1,2,518,151.5500,5,1,9,82
2,1,0,1,2.0000,1,2,518,151.5500,5,1,0,82
3,1,0,0,30.0000,1,2,518,151.5500,5,1,0,82
4,1,0,1,25.0000,1,2,518,151.5500,5,1,0,82


In [20]:
X = np.array(df.drop(['survived'], 1).astype(float))
X = preprocessing.scale(X)

/var/folders/rw/hql3pd214t59q_rc7_7h0lqm0000gn/T/ipykernel_86804/105476489.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(df.drop(['survived'], 1).astype(float))


In [21]:
y = np.array(df['survived'])

In [22]:
clf = MeanShift()
clf.fit(X)

MeanShift()

In [23]:
labels = clf.labels_
cluster_centers = clf.cluster_centers_

original_df['cluster_group'] = np.nan

for i in range(len(X)):
    original_df['cluster_group'].iloc[i] = labels[i]

n_clusters = len(np.unique(labels))

survival_rates = {}
for i in range(n_clusters):
    temp_df = original_df[(original_df['cluster_group'] == float(i))]
    survival_cluster = temp_df[(temp_df['survived'] == 1)]
    survival_rate = len(survival_cluster)/len(temp_df)
    survival_rates[i] = survival_rate

print(survival_rates)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


{0: 0.3782101167315175, 1: 0.9285714285714286, 2: 0.1}


In [24]:
print(original_df[(original_df['cluster_group'] == 1)])

     pclass  survived                                               name  \
17        1         1    Baxter, Mrs. James (Helene DeLaudeniere Chaput)   
49        1         1                 Cardeza, Mr. Thomas Drake Martinez   
50        1         1  Cardeza, Mrs. James Warburton Martinez (Charlo...   
97        1         1  Douglas, Mrs. Frederick Charles (Mary Helene B...   
111       1         1                     Fortune, Miss. Alice Elizabeth   
112       1         1                         Fortune, Miss. Ethel Flora   
113       1         1                         Fortune, Miss. Mabel Helen   
115       1         0                                  Fortune, Mr. Mark   
116       1         1                Fortune, Mrs. Mark (Mary McDougald)   
183       1         1                             Lesurer, Mr. Gustave J   
250       1         1                         Ryerson, Miss. Emily Borie   
251       1         1              Ryerson, Miss. Susan Parker "Suzette"   
253       1 

In [25]:
print(original_df[(original_df['cluster_group'] == 2)])

      pclass  survived                                               name  \
629        3         0                        Andersson, Mr. Anders Johan   
632        3         0  Andersson, Mrs. Anders Johan (Alfrida Konstant...   
644        3         0         Asplund, Mr. Carl Oscar Vilhelm Gustafsson   
646        3         1  Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...   
831        3         0                     Goodwin, Mr. Charles Frederick   
832        3         0            Goodwin, Mrs. Frederick (Augusta Tyler)   
1106       3         0             Panula, Mrs. Juha (Maria Emilia Ojala)   
1146       3         0               Rice, Mrs. William (Margaret Norton)   
1179       3         0                              Sage, Mr. John George   
1180       3         0                     Sage, Mrs. John (Annie Bullen)   

         sex   age  sibsp  parch    ticket     fare cabin embarked boat  \
629     male  39.0      1      5    347082  31.2750   NaN        S  NaN   
63

In [29]:
print(original_df[(original_df['cluster_group'] == 0)].describe())

            pclass     survived          age        sibsp        parch  \
count  1285.000000  1285.000000  1024.000000  1285.000000  1285.000000   
mean      2.303502     0.378210    29.696940     0.488716     0.326070   
std       0.832383     0.485129    14.404043     1.040865     0.676932   
min       1.000000     0.000000     0.166700     0.000000     0.000000   
25%       2.000000     0.000000    21.000000     0.000000     0.000000   
50%       3.000000     0.000000    28.000000     0.000000     0.000000   
75%       3.000000     1.000000    38.000000     1.000000     0.000000   
max       3.000000     1.000000    80.000000     8.000000     4.000000   

              fare        body  cluster_group  
count  1284.000000  119.000000         1285.0  
mean     29.966484  159.571429            0.0  
std      40.003574   97.302914            0.0  
min       0.000000    1.000000            0.0  
25%       7.895800   71.000000            0.0  
50%      14.054150  155.000000            0.0